In [ ]:
import json
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

actor_name_map = {}
movie_actor_map = {}
actor_genre_map = {}

with open("imdb_movies_2000to2022.prolific.json", "r") as df:
    for line in df:
        
        this_movie = json.loads(line)
                    
        # Add all actors to the id->name map
        for actor_id,actor_name in this_movie['actors']:
            actor_name_map[actor_id] = actor_name
            
        # For each actor, add this movie's genres to that actor's list
        for actor_id,actor_name in this_movie['actors']:
            this_actors_genres = actor_genre_map.get(actor_id, {})
            
            # Increment the count of genres for this actor
            for g in this_movie["genres"]:
                this_actors_genres[g] = this_actors_genres.get(g, 0) + 1
                
            actor_genre_map[actor_id] = this_actors_genres
    
        movie_actor_map[this_movie["imdb_id"]] = ({
            "movie": this_movie["title"],
            "actors": set([item[0] for item in this_movie['actors']]),
            "genres": this_movie["genres"]
        })

df = pd.DataFrame(actor_genre_map).T.fillna(0)

df


In [ ]:
points =[]
for k in range(2,32):
    lables = KMeans(n_clusters=k).fit_predict(df)
    score = silhouette_score(df,lables)
    points.append(score)

plt.plot(range(2 , 32) , points)
plt.xlabel('Number Of Clusters')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Scores for Different Values of k')
plt.show()

optimal_k = 10


kmeans_optimal = KMeans(n_clusters=optimal_k).fit_predict(df)


In [ ]:
import numpy as np

#Add the cluster labels as a column in the dataframe
df['cluster'] = kmeans_optimal
for cluster, actors in df.groupby('cluster'):
    print("Cluster:", cluster, "Size:", actors.shape[0])
    for a_id in actors.sample(5).index:
        print("\t", a_id, actor_name_map[a_id])